In [11]:
import pandas as pd
import os
import re
from sqlalchemy import create_engine
import json

## Data Loading

I need to load the akc data csv as a data frame, and then also build a list of dog breeds that we have in our categories in order to match the breeds we have in our project with the breeds y

In [2]:
akc_df = pd.read_csv("../../Dog_Breed_Files/Data/akc-data-latest.csv")

file_breeds = [breed.strip() for breed in akc_df["Unnamed: 0"].to_list()]

In [3]:
model_breeds = []

for folder_name in os.listdir("../../Dog_Breed_Files/Images/"):
    # each folder name has format of <string>-<breed>, search for first - and take everything
    search = re.search('.{9}-(.*)', folder_name)
    
    # clean each folder name (will replace some dashes we want at the end but will deal with that later)
    label = search.group(1).replace("_", " ")\
        .replace("-", " ")\
        .title()\
        .strip()
    
    model_breeds.append(label)

## Data Matching

In [4]:
breed_match = {}

for breed in model_breeds:
    if breed in file_breeds:
        breed_match[breed] = "match"
    else:
        breed_match[breed] = "no match"

breed_match_df = pd.DataFrame(data=breed_match.values(), index=breed_match.keys(), columns=["match"])

breed_match_df.to_clipboard()

In [5]:
# after a lot of trying to force things like "bassett" and "bassett hound" to match I ended up building the final name list in excel
name_match_df = pd.read_csv("folder_akc_name_match.csv")

## Building SQLite File

In [6]:
matches_df = name_match_df[name_match_df["AKC_Match"] != "none"]

In [9]:
final_df = pd.merge(left=matches_df, right=akc_df, left_on="AKC_Match", right_on="Unnamed: 0").set_index("folder_name")

final_df.head()

,cleaned_folder_name,AKC_Match,Unnamed: 0,description,temperament,popularity,min_height,max_height,min_weight,max_weight,...,grooming_frequency_value,grooming_frequency_category,shedding_value,shedding_category,energy_level_value,energy_level_category,trainability_value,trainability_category,demeanor_value,demeanor_category
folder_name,,,,,,,,,,,,,,,,,,,,,
Chihuahua,Chihuahua,Chihuahua,Chihuahua,"The Chihuahua is a balanced, graceful dog of t...","Charming, Graceful, Sassy",33,12.70,20.32,2.721554,2.721554,...,0.2,Occasional Bath/Brush,0.4,Occasional,0.6,Regular Exercise,0.4,Independent,0.6,Alert/Responsive
Maltese_dog,Maltese Dog,Maltese,Maltese,Maltese are affectionate toy dogs weighing les...,"Gentle, Playful, Charming",37,17.78,22.86,3.175147,3.175147,...,0.8,Daily Brushing,0.2,Infrequent,0.6,Regular Exercise,0.6,Agreeable,1.0,Outgoing
Pekinese,Pekinese,Pekingese,Pekingese,"Pekingese are compact, stocky toy dogs weighin...","Affectionate, Loyal, Regal in Manner",92,15.24,22.86,6.350293,6.350293,...,0.6,2-3 Times a Week Brushing,0.6,Seasonal,0.4,Calm,0.2,May be Stubborn,0.4,Reserved with Strangers
Shih-Tzu,Shih Tzu,Shih Tzu,Shih Tzu,Shih Tzu (pronounced in the West “sheed-zoo” o...,"Affectionate, Playful, Outgoing",20,22.86,26.67,4.082331,7.257478,...,0.8,Daily Brushing,0.2,Infrequent,0.6,Regular Exercise,0.6,Agreeable,0.8,Friendly
papillon,Papillon,Papillon,Papillon,"A tiny dog, measuring 8 to 11 inches at the sh...","Friendly, Alert, Happy",54,20.32,27.94,2.267962,4.535924,...,0.4,Weekly Brushing,0.6,Seasonal,0.6,Regular Exercise,1.0,Eager to Please,1.0,Outgoing


In [10]:
final_df.to_json("akc_data.json")

In [10]:
#create engine
engine = create_engine("sqlite:///akc_data.sqlite")

# send to sqlite
final_df.to_sql(name = 'akc_breed_data', con = engine, if_exists='replace', index_label="folder_name")